In [1]:
import numpy as np
import requests
import time

from pyspark import SQLContext, SparkContext
from pyspark import SparkConf

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.linalg import Vectors

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

In [2]:
# Load the dataset and labels
x=np.load('RMCx.npy')
y=np.load('RMCy.npy')

In [3]:
conf = SparkConf()
sc = SparkContext(conf = conf)
spark = SQLContext(sc)

In [4]:
# Calculate the rank of each feature
R=[]
for h in range(x.shape[1]):
    kmeans = KMeans(init='k-means++', n_clusters=np.unique(y).shape[0])
    ff=kmeans.fit_predict(x[:,h].reshape(-1,1))
    r=metrics.homogeneity_score(y,ff) #Use the homogeneity score as a rank of the feature
    R.append(r)

In [5]:
#Arrange feature accroding to thier ranks
Rnk=np.argsort(np.array(R))

In [6]:
#Initiate the cross-validation splitter
kfolds=StratifiedKFold(n_splits=5,shuffle=True)

In [7]:
#Per each set of ranks, use cross-validation to calculate accuracy.
smr=[]
et=0
lpa=np.zeros((0,2))
for j in range(Rnk.shape[0]):
    fd=x[:,Rnk[j:]]
    pp=0
    for train,test in kfolds.split(fd,y):
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[train],y[train].reshape(-1,1))))
        TrD = spark.createDataFrame(dff,schema=["label", "features"]).rdd.map(lambda row: LabeledPoint(row.label, MLLibVectors.fromML(row.features)))
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[test],y[test].reshape(-1,1))))
        TsD = spark.createDataFrame(dff,schema=["label", "features"]).rdd.map(lambda row: LabeledPoint(row.label, MLLibVectors.fromML(row.features)))
        model = RandomForest.trainClassifier(TrD, numClasses=np.unique(y).shape[0],categoricalFeaturesInfo={},numTrees=100) #The number of classes in the dataset
        predictions = model.predict(TsD.map(lambda x: x.features))
        st = time.time()
        labelsAndPredictions = TsD.map(lambda lp: lp.label).zip(predictions)
        lpa=np.vstack((lpa,labelsAndPredictions.toDF().toPandas()))
        et+=time.time()-st
        acc = labelsAndPredictions.filter(lambda lp: lp[0] == lp[1]).count() / float(TsD.count())
        pp=pp+acc
    pp=pp/kfolds.n_splits
    np.savetxt('F%d.csv'%j,lpa,delimiter=',')
    smr.append([j, pp, et*1000000/x.shape[0]]) #Calculate the time requires to predict a label per each object in uS.

In [8]:
smr

[[78.0, 0.9254, 6.9849],
 [77.0, 0.9257, 6.868],
 [76.0, 0.9254, 6.798],
 [75.0, 0.9255, 6.8025],
 [74.0, 0.9252, 6.8064],
 [73.0, 0.9254, 6.9112],
 [72.0, 0.9253, 6.7827],
 [71.0, 0.925, 6.7689],
 [70.0, 0.9251, 6.7854],
 [69.0, 0.9251, 6.8592],
 [68.0, 0.9253, 6.7573],
 [67.0, 0.9253, 6.7137],
 [66.0, 0.9256, 6.706],
 [65.0, 0.9251, 7.1871],
 [64.0, 0.9253, 7.2199],
 [63.0, 0.9254, 7.3147],
 [62.0, 0.9252, 6.9841],
 [61.0, 0.9253, 6.6506],
 [60.0, 0.9255, 6.7046],
 [59.0, 0.9254, 6.6921],
 [58.0, 0.9254, 6.6427],
 [57.0, 0.9254, 6.4835],
 [56.0, 0.9255, 6.5374],
 [55.0, 0.9258, 6.6122],
 [54.0, 0.9253, 6.5194],
 [53.0, 0.9256, 6.5272],
 [52.0, 0.9257, 6.4918],
 [51.0, 0.9253, 6.4434],
 [50.0, 0.9253, 6.4909],
 [49.0, 0.9256, 6.4413],
 [48.0, 0.9257, 6.4775],
 [47.0, 0.9252, 6.433],
 [46.0, 0.9252, 6.5239],
 [45.0, 0.9255, 6.4038],
 [44.0, 0.9254, 6.7169],
 [43.0, 0.9256, 6.4202],
 [42.0, 0.9255, 6.2888],
 [41.0, 0.9257, 6.3536],
 [40.0, 0.9255, 6.3336],
 [39.0, 0.9256, 6.332],
 [38.0